## 三大法人買賣超日報

In [11]:
from bs4 import BeautifulSoup as bs
from datetime import datetime, timedelta
import pandas as pd

In [12]:
url = 'http://www.twse.com.tw/ch/trading/fund/T86/T86.php'
payload={
'qdate':'105/05/13',
'select2':'24',
'sorting':'by_issue'    
}

In [13]:
def getTWDate(dt):
    year = int(dt.strftime('%Y')) - 1911
    monthdate = dt.strftime('%m/%d')
    ymd = '{}/{}'.format(year, monthdate)
    return ymd

def getTradingVolume(date):
    payload['qdate'] = getTWDate(date)
    res = requests.post(url, data=payload)    
    soup = bs(res.text)
    tb = soup.select('#tbl-sortable')[0]
    dfs = pd.read_html(tb.prettify('utf-8'), encoding='utf-8')
    stockdfs = dfs[0]
    stockdfs['date'] = date.date()
    return dfs[0]

In [14]:
import requests

currentdate = datetime.now()
dfall = []
for i in range(1,3):
    ymd = currentdate - timedelta(days = i)
    df = getTradingVolume(ymd)
    dfall.append(df)

C:\Anaconda2\lib\site-packages\bs4\__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [15]:
stockdf = pd.concat(dfall)

## SQLite 操作
不用安裝好像也可以直接用

In [3]:
import sqlite3 as lite
con = lite.connect('finance2.sqlite')
con.close()

In [6]:
import sqlite3 as lite
con = lite.connect('finance2.sqlite')
cur = con.cursor()
cur.execute('SELECT SQLITE_VERSION()')
data = cur.fetchone()
print data
con.close()

(u'3.6.21',)


In [7]:
import sqlite3 as lite
con = lite.connect('finance2.sqlite')
cur = con.cursor()

cur.execute("DROP TABLE IF EXISTS PhoneAddress") 
cur.execute("CREATE TABLE PhoneAddress(phone CHAR(10) PRIMARY KEY, address TEXT, name TEXT unique, age INT NOT NULL)") 
cur.execute("INSERT INTO PhoneAddress VALUES('0912173381','United State','Jhon Doe',53)") 
cur.execute("INSERT INTO PhoneAddress VALUES('0928375018','Tokyo Japan','MuMu Cat',6)") 
cur.execute("INSERT INTO PhoneAddress VALUES('0957209108','Taipei','Richard',29)") 
con.commit()
cur.execute("SELECT phone,address FROM PhoneAddress") 

data = cur.fetchall()
print data
con.close()

[(u'0912173381', u'United State'), (u'0928375018', u'Tokyo Japan'), (u'0957209108', u'Taipei')]


In [8]:
import sqlite3 as lite
con = lite.connect('finance2.sqlite')
cur = con.cursor()

cur.execute("SELECT phone,address FROM PhoneAddress") 

data = cur.fetchone()
print data[0], data[1]

data = cur.fetchall()
print data
con.close()

0912173381 United State
[(u'0928375018', u'Tokyo Japan'), (u'0957209108', u'Taipei')]


In [9]:
a = (1,2) #tuple方法比較少
b = [1,2]
dir(b)

['__add__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__delslice__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getslice__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__imul__',
 '__init__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__rmul__',
 '__setattr__',
 '__setitem__',
 '__setslice__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'append',
 'count',
 'extend',
 'index',
 'insert',
 'pop',
 'remove',
 'reverse',
 'sort']

In [16]:
stockdf.head()

,證券 代號,證券 名稱,外資 買進股數,外資 賣出股數,外資 買賣超股數,投信 買進股數,投信 賣出股數,投信 買賣超股數,自營商 買賣超股數,自營商 買進股數 (自行買賣),自營商 賣出股數 (自行買賣),自營商 買賣超股數 (自行買賣),自營商 買進股數 (避險),自營商 賣出股數 (避險),自營商 買賣超股數 (避險),三大法人 買賣超股數,date
0,2408.0,南亞科,18079000.0,4097000.0,13982000.0,0.0,0.0,0.0,329000.0,317000.0,17000.0,300000.0,154000.0,125000.0,29000.0,14311000.0,2016-05-13
1,3474.0,華亞科,9560000.0,4665000.0,4895000.0,0.0,0.0,0.0,113000.0,201000.0,30000.0,171000.0,0.0,58000.0,-58000.0,5008000.0,2016-05-13
2,2449.0,京元電,8513011.0,4469755.0,4043256.0,58000.0,490000.0,-432000.0,472000.0,0.0,107000.0,-107000.0,1266000.0,687000.0,579000.0,4083256.0,2016-05-13
3,3413.0,京鼎,529000.0,24000.0,505000.0,0.0,0.0,0.0,-1000.0,0.0,1000.0,-1000.0,0.0,0.0,0.0,504000.0,2016-05-13
4,3016.0,嘉晶,440000.0,91000.0,349000.0,0.0,0.0,0.0,-12000.0,2000.0,14000.0,-12000.0,0.0,0.0,0.0,337000.0,2016-05-13


In [17]:
import sqlite3 as lite
con = lite.connect('finance2.sqlite')
stockdf.to_sql(name = 'trading_volume', index=False, con = con, if_exists="replace")
#con.commit()
con.close()

C:\Anaconda2\lib\site-packages\pandas\core\generic.py:1160: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  chunksize=chunksize, dtype=dtype)


In [18]:
#stockdf.head()
#len(stockdf.columns)
stockdf.columns = ['id', 'name', 'buy', 'sale', '4','5', '6', '7' '8','9','10','11','12', '13', '14', '15', '16', 'date']

In [19]:
with lite.connect('finance2.sqlite') as con:
    stockdf.to_sql(name = 'trading_volume', index=False, con = con, if_exists="replace")

In [20]:
import pandas as pd
with lite.connect('finance2.sqlite') as con:
    df2 = pd.read_sql_query('select * from trading_volume', con)
df2.head()

,id,name,buy,sale,4,5,6,78,9,10,11,12,13,14,15,16,date
0,2408.0,南亞科,18079000.0,4097000.0,13982000.0,0.0,0.0,0.0,329000.0,317000.0,17000.0,300000.0,154000.0,125000.0,29000.0,14311000.0,2016-05-13
1,3474.0,華亞科,9560000.0,4665000.0,4895000.0,0.0,0.0,0.0,113000.0,201000.0,30000.0,171000.0,0.0,58000.0,-58000.0,5008000.0,2016-05-13
2,2449.0,京元電,8513011.0,4469755.0,4043256.0,58000.0,490000.0,-432000.0,472000.0,0.0,107000.0,-107000.0,1266000.0,687000.0,579000.0,4083256.0,2016-05-13
3,3413.0,京鼎,529000.0,24000.0,505000.0,0.0,0.0,0.0,-1000.0,0.0,1000.0,-1000.0,0.0,0.0,0.0,504000.0,2016-05-13
4,3016.0,嘉晶,440000.0,91000.0,349000.0,0.0,0.0,0.0,-12000.0,2000.0,14000.0,-12000.0,0.0,0.0,0.0,337000.0,2016-05-13


In [21]:
df2.columns

Index([u'id', u'name', u'buy', u'sale', u'4', u'5', u'6', u'78', u'9', u'10',
       u'11', u'12', u'13', u'14', u'15', u'16', u'date'],
      dtype='object')

In [22]:
import pandas as pd
with lite.connect('finance2.sqlite') as con:
    df2 = pd.read_sql_query(\
        u"select id, name from trading_volume where buy > 307000", con)
df2

,id,name
0,2408.0,南亞科
1,3474.0,華亞科
2,2449.0,京元電
3,3413.0,京鼎
4,3016.0,嘉晶
5,8016.0,矽創
6,2401.0,凌陽
7,2388.0,威盛
8,2337.0,旺宏
9,2379.0,瑞昱


In [25]:
df2.to_excel('test.xlsx')#匯出EXECEL
#tab鍵可以提示
#alt+enter 執行並增加一行

In [27]:
df3 = pd.read_excel('test.xlsx')

In [28]:
df3

,id,name
0,2408,南亞科
1,3474,華亞科
2,2449,京元電
3,3413,京鼎
4,3016,嘉晶
5,8016,矽創
6,2401,凌陽
7,2388,威盛
8,2337,旺宏
9,2379,瑞昱


## 2330　102年第1季 IFRS 合併財務報表預覽

In [29]:
import requests
url = 'http://mops.twse.com.tw/server-java/t164sb01'
payload = {
'step':'1',
'CO_ID':'2330',
'SYEAR':'2013',
'SSEASON':'1',
'REPORT_ID':'C'
}
res = requests.post(url, data= payload)
print res.encoding
res.encoding = 'big5'
print res.encoding

ISO-8859-1
big5


In [32]:
from bs4 import BeautifulSoup as bs
soup = bs(res.text)
tb = soup.select('.result_table')[0]
#print tb


In [33]:
import pandas as pd
dfs = pd.read_html(tb.prettify('utf-8'), encoding = 'utf-8', header=0, skiprows=0)

In [34]:
stockdf = dfs[0]
stockdf['stockno'] = '2330'
import sqlite3 as lite
with lite.connect('finance2.sqlite') as con:
    stockdf.to_sql('ifrs', con = con, if_exists='replace')

## 三大法人買賣明細資訊

In [ ]:
Document(Doc):
requests    ------------> document: <html> data </html>
100 * 3
    
AJAX(XHR):
requests    ------------> document: <html></html>
requests    ------------> javascript: $('html').html(data)
requests    ------------> data: data
2 + 1 + 100 * 1

In [ ]:
XML:
<employee>
    <name> qoo </name>
    <age> 10 <age>
</employee>

JSON:
{employee:{name:qoo, age:10}}

In [35]:
import requests
res = requests.get('http://www.tpex.org.tw/web/stock/3insti/daily_trade/3itrade_hedge_result.php?l=zh-tw&se=EW&t=D&d=105/05/10&_=1463283795803')
print res

<Response [200]>


In [36]:
#print res.text
import json
jd = json.loads(res.text)
#print jd['aaData']

In [37]:
import pandas as pd
df = pd.DataFrame(jd['aaData'])
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,006201,寶富櫃,0,0,0,0,0,0,"-45,000",0,0,0,0,"45,000","-45,000","-45,000"
1,1258,F-其祥,"3,000",0,"3,000",0,0,0,"-1,000",0,"1,000","-1,000",0,0,0,"2,000"
2,1259,安心,"14,000",0,"14,000",0,0,0,0,0,0,0,0,0,0,"14,000"
3,1264,德麥,"4,000","1,000","3,000",0,0,0,0,0,0,0,0,0,0,"3,000"
4,1565,精華,"148,000","136,000","12,000",0,0,0,"-1,000",0,0,0,"1,000","2,000","-1,000","11,000"
